In [ ]:
!pip install kaggle

In [2]:
!pip install opendatasets

In [3]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/issaisasank/guns-object-detection/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: kiviyiyi
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/issaisasank/guns-object-detection


In [ ]:
! pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

# Лабораторная работа №8

## Выполнил Ворошилов Кирилл, М8О-401Б-21


### 1. Выбор начальных условий
* для распознавания будет использован датасет Guns Object Detection

In [ ]:
import os
import torch
from torchvision.datasets import CIFAR10
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from ultralytics import YOLO
from PIL import Image
from sklearn.metrics import classification_report, precision_score, recall_score, average_precision_score
import numpy as np

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

Используемое устройство: cpu


In [ ]:
import os
from PIL import Image

LABELS_PATH = "/content/guns-object-detection/Labels"
IMAGES_PATH = "/content/guns-object-detection/Images"

for i in range(1, len(os.listdir(IMAGES_PATH)) + 1):
    img_path = os.path.join(IMAGES_PATH, f"{i}.jpeg")
    label_path = os.path.join(LABELS_PATH, f"{i}.txt")

    if not os.path.exists(img_path) or not os.path.exists(label_path):
        continue

    with Image.open(img_path) as img:
        W, H = img.size

    with open(label_path, 'r') as f:
        lines = f.readlines()

    if not lines:
        continue

    new_lines = []
    try:
        box_count = int(lines[0].strip())
    except ValueError:
        continue

    for line in lines[1:1 + box_count]:
        x_min, y_min, x_max, y_max = map(int, line.strip().split())

        x_center = ((x_min + x_max) / 2) / W
        y_center = ((y_min + y_max) / 2) / H
        width = (x_max - x_min) / W
        height = (y_max - y_min) / H

        new_lines.append(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    with open(label_path, 'w') as f:
        f.writelines(new_lines)

In [ ]:
from sklearn.model_selection import train_test_split
import shutil
import os

images_dir = "/content/guns-object-detection/Images"
labels_dir = "/content/guns-object-detection/Labels"

indices = [os.path.splitext(f)[0] for f in os.listdir(images_dir) if f.endswith('.jpeg')]
train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)

for split in ['train', 'val']:
    os.makedirs(f"{images_dir}/{split}", exist_ok=True)
    os.makedirs(f"{labels_dir}/{split}", exist_ok=True)

for idx in train_idx:
    if os.path.exists(f"{images_dir}/{idx}.jpeg") and os.path.exists(f"{labels_dir}/{idx}.txt"):
        shutil.move(f"{images_dir}/{idx}.jpeg", f"{images_dir}/train/{idx}.jpeg")
        shutil.move(f"{labels_dir}/{idx}.txt", f"{labels_dir}/train/{idx}.txt")

for idx in val_idx:
    if os.path.exists(f"{images_dir}/{idx}.jpeg") and os.path.exists(f"{labels_dir}/{idx}.txt"):
        shutil.move(f"{images_dir}/{idx}.jpeg", f"{images_dir}/val/{idx}.jpeg")
        shutil.move(f"{labels_dir}/{idx}.txt", f"{labels_dir}/val/{idx}.txt")


In [ ]:
import shutil
import os

base_path = "/content/guns-object-detection"

for split in ['train', 'val']:
    label_dir = os.path.join(base_path, 'Labels', split)
    image_dir = os.path.join(base_path, 'Images', split)

    for fname in os.listdir(label_dir):
        if fname.endswith(".txt"):
            src = os.path.join(label_dir, fname)
            dst = os.path.join(image_dir, fname)
            shutil.move(src, dst)



In [ ]:
yaml_path = "/content/guns-object-detection/guns_dataset.yaml"
yaml_content = """path: /content/guns-object-detection
train: Images/train
val: Images/val
nc: 1
names: ['gun']


"""
with open(yaml_path, "w") as f:
    f.write(yaml_content)

### 2. Создание бейзлайна и оценка качества

Обучим модели из torchvision

In [ ]:
yolo_model_base = YOLO("yolov8n.yaml")

In [ ]:
yolo_model_base.train(data=yaml_path, epochs=50, imgsz=640, batch=32, name="yolo_base")

Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/guns-object-detection/guns_dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_base, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True,

100%|██████████| 755k/755k [00:00<00:00, 14.4MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 18                  -1  1    123648  ultralytics.nn.modules.block.C2f             [192, 128, 1]                 
 19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  1    493056  ultralytics.nn.modules.block.C2f             [384, 256, 1]                 
 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
YOLOv8n summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Freezing layer 'model.22.dfl.conv.weight'
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 81.4±39.6 MB/s, size: 7.5 KB)


train: Scanning /content/guns-object-detection/Images/train... 266 images, 0 backgrounds, 0 corrupt: 100%|██████████| 266/266 [00:00<00:00, 1395.93it/s]


...

                   all         67         78      0.578      0.397      0.471      0.206



50 epochs completed in 3.554 hours.
Optimizer stripped from runs/detect/yolo_base/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolo_base/weights/best.pt, 6.2MB

Validating runs/detect/yolo_base/weights/best.pt...
Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:18<00:00,  9.06s/it]


                   all         67         78       0.58      0.397      0.472      0.207
Speed: 7.3ms preprocess, 256.5ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/yolo_base


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ba04bf6e410>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

Результат:

In [ ]:
results_val_base = yolo_model_base.val(data=yaml_path, iou=0.5)
print("\n[YOLOv8 BASELINE Evaluation Metrics]")
print("Precision:", results_val_base.box.p)
print("Recall:", results_val_base.box.r)
print("mAP@0.5:", results_val_base.box.map50)
print("mAP@0.5:0.95:", results_val_base.box.map)

Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 258.2±115.1 MB/s, size: 5.5 KB)


val: Scanning /content/guns-object-detection/Images/val.cache... 67 images, 0 backgrounds, 0 corrupt: 100%|██████████| 67/67 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:13<00:00,  4.42s/it]


                   all         67         78       0.53      0.385      0.487      0.204
Speed: 1.6ms preprocess, 186.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/yolo_base2

[YOLOv8 BASELINE Evaluation Metrics]
Precision: [     0.5302]
Recall: [    0.38462]
mAP@0.5: 0.48736101642431084
mAP@0.5:0.95: 0.20421658516620161


### 3. Улучшение бейзлайна

Визуализируем данные

In [ ]:
import matplotlib as plt

In [ ]:
from torchvision.io import read_image
BASE_IMAGE_PATH = "/content/guns-object-detection/Images"
BASE_LABELS_PATH = "/content/guns-object-detection/Labels"

image_data = []
image_labels = []

image_count = len(os.listdir(BASE_IMAGE_PATH))

for i in range(1, image_count + 1):
    image_path = os.path.join(BASE_IMAGE_PATH, f"{i}.jpeg")
    if not os.path.exists(image_path):
        continue
    img = read_image(image_path)
    image_data.append(img)

    _, H, W = img.shape
    label_mask = torch.zeros((H, W), dtype=torch.float32)

    label_path = os.path.join(BASE_IMAGE_PATH, f"{i}.txt")
    if os.path.exists(label_path):
        with open(label_path, 'r') as label_file:
            l_count = int(label_file.readline().strip())
            for j in range(l_count):
                box = list(map(int, label_file.readline().split()))
                label_mask[box[1]:box[3], box[0]:box[2]] = 1.0

    image_labels.append(label_mask)

plt.subplots(nrows=2, ncols=4, figsize=(20, 10))

for idx, img_index in enumerate(np.random.randint(0, len(image_data), size=4)):
    plt.subplot(2, 4, idx + 1)
    plt.axis('off')
    plt.title(f'Image: {img_index + 1}')
    img_np = image_data[img_index].permute(1, 2, 0).numpy()
    plt.imshow(img_np)

    plt.subplot(2, 4, idx + 5)
    plt.axis('off')
    plt.title(f'Label mask: {img_index + 1}')
    plt.imshow(image_labels[img_index].numpy(), cmap='gray')

plt.show()

AttributeError: module 'matplotlib' has no attribute 'subplots'

Проведем препроцессинг данных

In [ ]:
yolo_model_improved = YOLO("yolov8n.yaml")

Улучшим модель подобрав гиперпараметры

In [ ]:
results = yolo_model_improved.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    mosaic=1.0,
    mixup=0.0,
    batch=32,
    name="yolo_improved"
)

Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/guns-object-detection/guns_dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_improved2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=

train: Scanning /content/guns-object-detection/Images/train.cache... 266 images, 0 backgrounds, 0 corrupt: 100%|██████████| 266/266 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 11.0±14.6 MB/s, size: 7.1 KB)


val: Scanning /content/guns-object-detection/Images/val.cache... 67 images, 0 backgrounds, 0 corrupt: 100%|██████████| 67/67 [00:00<?, ?it/s]

Plotting labels to runs/detect/yolo_improved2/labels.jpg... 


...

      50/50         0G       1.81      2.003      2.321         12        640: 100%|██████████| 9/9 [03:47<00:00, 25.26s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:17<00:00,  8.99s/it]

                   all         67         78      0.578      0.397      0.471      0.206



50 epochs completed in 3.423 hours.
Optimizer stripped from runs/detect/yolo_improved2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/yolo_improved2/weights/best.pt, 6.2MB

Validating runs/detect/yolo_improved2/weights/best.pt...
Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:15<00:00,  7.95s/it]


                   all         67         78       0.58      0.397      0.472      0.207
Speed: 2.6ms preprocess, 228.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/yolo_improved2


In [ ]:
results_val_improved = yolo_model_improved.val(data=yaml_path, iou=0.5)
print("\n[YOLOv8 IMPROVED Evaluation Metrics]")
print("Precision:", results_val_improved.box.p)
print("Recall:", results_val_improved.box.r)
print("mAP@0.5:", results_val_improved.box.map50)
print("mAP@0.5:0.95:", results_val_improved.box.map)

Ultralytics 8.3.130 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 264.0±128.6 MB/s, size: 5.5 KB)


val: Scanning /content/guns-object-detection/Images/val.cache... 67 images, 0 backgrounds, 0 corrupt: 100%|██████████| 67/67 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:12<00:00,  4.27s/it]


                   all         67         78       0.53      0.385      0.487      0.204
Speed: 1.6ms preprocess, 181.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/yolo_improved22

[YOLOv8 IMPROVED Evaluation Metrics]
Precision: [     0.5302]
Recall: [    0.38462]
mAP@0.5: 0.48736101642431084
mAP@0.5:0.95: 0.20421658516620161


### 4. Имплементация алгоритма машинного обучения

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T

class GunDetectionDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
      img_name = self.images[idx]
      img_path = os.path.join(self.image_dir, img_name)
      base = os.path.splitext(img_name)[0]
      label_path = os.path.join(self.label_dir, base + '.txt')

      image = Image.open(img_path).convert('RGB')
      if self.transform:
          image = self.transform(image)

      boxes = []
      if os.path.exists(label_path):
          try:
              with open(label_path, 'r', encoding='utf-8', errors='ignore') as f:
                  for line in f.readlines():
                      parts = line.strip().split()
                      if len(parts) == 5:
                          cls, x, y, w, h = map(float, parts)
                          boxes.append([cls, x, y, w, h])
          except Exception as e:
              print(f"Ошибка при чтении {label_path}: {e}")

      boxes = torch.tensor(boxes) if boxes else torch.zeros((0, 5))
      return image, boxes

def detection_collate_fn(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    return torch.stack(images), targets


Имплементируем алгоритмы:

In [ ]:
import torch.nn as nn
import torch.optim as optim

class TinyYOLO(nn.Module):
    def __init__(self):
        super(TinyYOLO, self).__init__()
        self.backbone = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(256, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(512, 1024, 3, padding=1), nn.BatchNorm2d(1024), nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024, 512), nn.ReLU(),
            nn.Linear(512, 128), nn.ReLU(),
            nn.Linear(128, 5)
        )

    def forward(self, x):
        return self.head(self.backbone(x))

In [ ]:
import torch
import torch.nn as nn

class SimpleYOLOLoss(nn.Module):
    def __init__(self, class_weight=3.0):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss()
        self.l1 = nn.L1Loss()
        self.class_weight = class_weight

    def forward(self, preds, targets):
        loss = 0.0
        valid = 0

        for pred, target in zip(preds, targets):
            if target.size(0) == 0:
                continue

            valid += 1
            target_obj = target[0]

            bbox_pred = pred[:4]
            bbox_target = target_obj[1:5]

            bbox_loss = self.l1(bbox_pred, bbox_target)

            class_target = torch.tensor([1.0], device=pred.device)
            class_logit = pred[4:]

            class_loss = self.bce(class_logit, class_target)

            total = bbox_loss + self.class_weight * class_loss
            loss += total

        if valid == 0:
            return torch.tensor(0.0, requires_grad=True, device=preds.device)

        return loss / valid

In [ ]:
def train(model, loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, targets in loader:
            images = images.to(device)
            targets = [t.to(device) for t in targets]

            preds = model(images)
            loss = criterion(preds, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}: Loss = {total_loss / len(loader):.4f}")

def run_experiment(train_ds, test_ds, batch_size, lr, epochs):
    print(f"batch_size={batch_size}, lr={lr}, epochs={epochs}")
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=detection_collate_fn)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, collate_fn=detection_collate_fn)

    model = TinyYOLO().to(device)
    criterion = SimpleYOLOLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train(model, train_loader, optimizer, criterion, epochs)
    evaluate(model, test_loader)


In [ ]:
from sklearn.metrics import precision_score, recall_score, average_precision_score
import torch

def evaluate(model, loader, iou_threshold=0.2):
    model.eval()
    all_preds = []
    all_targets = []

    def iou(box1, box2):
        def to_xyxy(box):
            x1 = box[0] - box[2] / 2
            y1 = box[1] - box[3] / 2
            x2 = box[0] + box[2] / 2
            y2 = box[1] + box[3] / 2
            return x1, y1, x2, y2

        x1_1, y1_1, x2_1, y2_1 = to_xyxy(box1)
        x1_2, y1_2, x2_2, y2_2 = to_xyxy(box2)

        inter_x1 = max(x1_1, x1_2)
        inter_y1 = max(y1_1, y1_2)
        inter_x2 = min(x2_1, x2_2)
        inter_y2 = min(y2_1, y2_2)

        inter_area = max(inter_x2 - inter_x1, 0) * max(inter_y2 - inter_y1, 0)
        area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
        area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
        union_area = area1 + area2 - inter_area

        return inter_area / union_area if union_area > 0 else 0.0

    with torch.no_grad():
        for images, targets in loader:
            images = images.to(model.device if hasattr(model, "device") else "cuda" if torch.cuda.is_available() else "cpu")
            preds = model(images).cpu()

            for pred, gt_boxes in zip(preds, targets):
                pred_box = pred[:4]
                pred_conf = torch.sigmoid(pred[4])
                matched = False
                for gt in gt_boxes:
                    gt_box = gt[1:5]
                    if iou(pred_box, gt_box) >= iou_threshold:
                        matched = True
                        break

                pred_cls = int(pred_conf > 0.5)
                target_cls = int(matched)

                all_preds.append(pred_conf.item())
                all_targets.append(target_cls)

    binary_preds = [int(p > 0.5) for p in all_preds]
    print("[Evaluation]")
    print("Precision:", precision_score(all_targets, binary_preds, zero_division=0))
    print("Recall:", recall_score(all_targets, binary_preds, zero_division=0))
    print("mAP (approx):", average_precision_score(all_targets, all_preds))


Обучим модель на базовом бейзлайне

In [ ]:
transform = T.Compose([
    T.Resize((128, 128)),
    T.ToTensor(),
])

train_dataset = GunDetectionDataset(
    image_dir='/content/guns-object-detection/Images/train',
    label_dir='/content/guns-object-detection/Images/train',
    transform=transform
)

val_dataset = GunDetectionDataset(
    image_dir='/content/guns-object-detection/Images/val',
    label_dir='/content/guns-object-detection/Images/val',
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=detection_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=detection_collate_fn)


In [87]:
model_b = TinyYOLO().to(device)
criterion = SimpleYOLOLoss()
optimizer = optim.Adam(model_b.parameters(), lr=1e-3)

In [88]:
train(model_b, train_loader, optimizer, criterion, 50)

Epoch 1: Loss = 0.6462
Epoch 2: Loss = 0.2021
Epoch 3: Loss = 0.1811
Epoch 4: Loss = 0.1684
Epoch 5: Loss = 0.1739
Epoch 6: Loss = 0.1525
Epoch 7: Loss = 0.1458
Epoch 8: Loss = 0.1466
Epoch 9: Loss = 0.1387
Epoch 10: Loss = 0.1481
Epoch 11: Loss = 0.1508
Epoch 12: Loss = 0.1437
Epoch 13: Loss = 0.1264
Epoch 14: Loss = 0.1208
Epoch 15: Loss = 0.1196
Epoch 16: Loss = 0.1192
Epoch 17: Loss = 0.1118
Epoch 18: Loss = 0.1121
Epoch 19: Loss = 0.1052
Epoch 20: Loss = 0.1285
Epoch 21: Loss = 0.1141
Epoch 22: Loss = 0.1050
Epoch 23: Loss = 0.1066
Epoch 24: Loss = 0.1087
Epoch 25: Loss = 0.1040
Epoch 26: Loss = 0.1040
Epoch 27: Loss = 0.0934
Epoch 28: Loss = 0.0880
Epoch 29: Loss = 0.0922
Epoch 30: Loss = 0.0901
Epoch 31: Loss = 0.0974
Epoch 32: Loss = 0.0861
Epoch 33: Loss = 0.0841
Epoch 34: Loss = 0.0769
Epoch 35: Loss = 0.0817
Epoch 36: Loss = 0.0814
Epoch 37: Loss = 0.0816
Epoch 38: Loss = 0.0742
Epoch 39: Loss = 0.0852
Epoch 40: Loss = 0.0718
Epoch 41: Loss = 0.0754
Epoch 42: Loss = 0.0757
E

In [104]:
evaluate(model_b, val_loader)

[Evaluation]
Precision: 0.29850746268656714
Recall: 1.0
mAP (approx): 0.43488602235909724


Обучим модель с улучшенным бейзланом

In [ ]:
aug_transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    T.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    T.ToTensor(),
])

train_dataset_aug = GunDetectionDataset(
    image_dir='/content/guns-object-detection/Images/train',
    label_dir='/content/guns-object-detection/Images/train',
    transform=aug_transform
)

val_dataset_aug = GunDetectionDataset(
    image_dir='/content/guns-object-detection/Images/val',
    label_dir='/content/guns-object-detection/Images/val',
    transform=T.Compose([
        T.Resize((224, 224)),
        T.ToTensor()
    ])
)

train_loader_aug = DataLoader(train_dataset_aug, batch_size=8, shuffle=True, collate_fn=detection_collate_fn)
val_loader_aug = DataLoader(val_dataset_aug, batch_size=8, shuffle=False, collate_fn=detection_collate_fn)


In [106]:
tiny_improved = TinyYOLO().to(device)

In [107]:
criterion = SimpleYOLOLoss()
optimizer = optim.Adam(tiny_improved.parameters(), lr=5e-4)

In [108]:
train(tiny_improved, train_loader_aug, optimizer, criterion, epochs=50)

Epoch 1: Loss = 0.4153
Epoch 2: Loss = 0.1718
Epoch 3: Loss = 0.1729
Epoch 4: Loss = 0.1657
Epoch 5: Loss = 0.1632
Epoch 6: Loss = 0.1542
Epoch 7: Loss = 0.1541
Epoch 8: Loss = 0.1497
Epoch 9: Loss = 0.1461
Epoch 10: Loss = 0.1499
Epoch 11: Loss = 0.1488
Epoch 12: Loss = 0.1499
Epoch 13: Loss = 0.1467
Epoch 14: Loss = 0.1531
Epoch 15: Loss = 0.1441
Epoch 16: Loss = 0.1471
Epoch 17: Loss = 0.1435
Epoch 18: Loss = 0.1387
Epoch 19: Loss = 0.1485
Epoch 20: Loss = 0.1398
Epoch 21: Loss = 0.1411
Epoch 22: Loss = 0.1436
Epoch 23: Loss = 0.1347
Epoch 24: Loss = 0.1339
Epoch 25: Loss = 0.1374
Epoch 26: Loss = 0.1325
Epoch 27: Loss = 0.1301
Epoch 28: Loss = 0.1302
Epoch 29: Loss = 0.1357
Epoch 30: Loss = 0.1356
Epoch 31: Loss = 0.1324
Epoch 32: Loss = 0.1381
Epoch 33: Loss = 0.1318
Epoch 34: Loss = 0.1302
Epoch 35: Loss = 0.1276
Epoch 36: Loss = 0.1300
Epoch 37: Loss = 0.1318
Epoch 38: Loss = 0.1316
Epoch 39: Loss = 0.1301
Epoch 40: Loss = 0.1316
Epoch 41: Loss = 0.1279
Epoch 42: Loss = 0.1290
E

In [109]:
evaluate(tiny_improved, val_loader_aug)

[Evaluation]
Precision: 0.3880597014925373
Recall: 1.0
mAP (approx): 0.40395304029754886


## Выводы

- **Идентичность метрик:** Улучшенная версия YOLOv8 показала идентичные значения метрик по сравнению с базовой моделью. Это указывает на отсутствие значимого влияния модификаций на такие показатели, как Precision, Recall и mAP.
- **Особенности кастомной реализации:** Кастомный подход продемонстрировал высокую полноту (Recall = 1.0), но при этом более низкую точность (Precision), что может свидетельствовать о переобобщении или завышенной чувствительности модели.
- **Сравнение по mAP:** Несмотря на снижение точности, значения mAP (в частности, mAP@0.5) для кастомной оценки находятся на уровне, сопоставимом с YOLOv8, что подчёркивает потенциал данного подхода при дальнейшем развитии.
- **Значение параметров:** Сравнительный анализ подтверждает важность многогранной оценки модели с использованием различных метрик и подчёркивает необходимость тонкой настройки гиперпараметров.
